In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score,\
                                    GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor,\
                             RandomForestRegressor

In [58]:
covid = pd.read_csv('../data/covid19_nyt_us_counties.csv')

In [50]:
print(covid.shape)
covid.head()

(1384683, 6)


,date,county,state_name,county_fips_code,confirmed_cases,deaths
0,2020-07-22,Kent,Rhode Island,44003.0,1351,101.0
1,2020-07-22,Providence,Rhode Island,44007.0,14033,801.0
2,2020-07-23,Apache,Arizona,4001.0,2841,117.0
3,2020-07-23,Coconino,Arizona,4005.0,2778,110.0
4,2020-07-23,Maricopa,Arizona,4013.0,102247,1651.0


In [51]:
len(covid.county_fips_code.unique())

3219

In [52]:
census = pd.read_csv('../data/census_bureau_acs_county_2018_5yr.csv')

In [53]:
print(census.shape)
census.head()

(3220, 242)


,geo_id,do_date,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,...,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english
0,35039,2014-01-01,39307.0,12398.0,19250.0,20057.0,40.6,1224.0,1240.0,1278.0,...,NaN,NaN,2170.0,1985.0,1970.0,9260.0,1653.0,NaN,NaN,NaN
1,72133,2014-01-01,22066.0,7465.0,10650.0,11416.0,37.8,567.0,662.0,864.0,...,2171.0,1242.0,1155.0,1362.0,1319.0,5884.0,1410.0,NaN,NaN,NaN
2,72043,2014-01-01,39265.0,13346.0,19056.0,20209.0,40.7,945.0,1109.0,1431.0,...,3023.0,2529.0,1851.0,2144.0,1901.0,9475.0,2219.0,NaN,NaN,NaN
3,72151,2014-01-01,34149.0,11722.0,16541.0,17608.0,42.5,754.0,1095.0,1020.0,...,1969.0,1700.0,1957.0,1388.0,1933.0,8055.0,2147.0,NaN,NaN,NaN
4,72071,2014-01-01,42420.0,15012.0,20629.0,21791.0,41.6,898.0,1159.0,1381.0,...,3246.0,2725.0,1996.0,2057.0,2211.0,10099.0,2551.0,NaN,NaN,NaN


In [54]:
covid.groupby('county_fips_code').max().isna().sum()

date                0
county              0
state_name          0
confirmed_cases     0
deaths             78
dtype: int64

In [55]:
by_county = covid.groupby('county_fips_code').max()

In [10]:
census.set_index('geo_id', inplace=True)

In [12]:
df = pd.merge(by_county, census, left_index = True, right_index=True)

In [13]:
print('by_county df:', by_county.shape)
print('census df:', census.shape)
print('final df:', df.shape)

by_county df: (3218, 5)
census df: (3220, 241)
final df: (3211, 246)


In [16]:
for col_name in df.columns:
    if 'poverty' in col_name:
        print(col_name)

pop_determined_poverty_status
poverty


In [18]:
df['poverty_rate'] = df.poverty / df.pop_determined_poverty_status

df['death_rate'] = df.deaths / df.confirmed_cases

looks like all the nans in the deaths column are from puerto rico

In [21]:
df.dropna(subset=['deaths'], inplace=True)

In [22]:
hosp_county = pd.read_csv('../data/us_healthcare_capacity-county-CovidCareMap.csv')

In [23]:
print(hosp_county.shape)
hosp_county.head()

(2537, 22)


,fips_code,State,County Name,Staffed All Beds,Staffed ICU Beds,Licensed All Beds,All Bed Occupancy Rate,ICU Bed Occupancy Rate,Population,Population (20+),...,Staffed All Beds [Per 1000 Adults (20+)],Staffed All Beds [Per 1000 Elderly (65+)],Staffed ICU Beds [Per 1000 People],Staffed ICU Beds [Per 1000 Adults (20+)],Staffed ICU Beds [Per 1000 Elderly (65+)],Licensed All Beds [Per 1000 People],Licensed All Beds [Per 1000 Adults (20+)],Licensed All Beds [Per 1000 Elderly (65+)],ICU Bed Source,ICU Bed Source Last Updated
0,2016,AK,Aleutians West,0.0,0.0,0.0,NaN,NaN,5723.0,4983.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,Facility aggregation,NaN
1,2020,AK,Anchorage,979.0,73.0,1149.0,0.66,0.71,291538.0,213508.0,...,4.585,30.279,0.250,0.342,2.258,3.941,5.382,35.536,Facility aggregation,NaN
2,2050,AK,Bethel,34.0,0.0,37.0,0.40,NaN,18216.0,11082.0,...,3.068,24.251,0.000,0.000,0.000,2.031,3.339,26.391,Facility aggregation,NaN
3,2070,AK,Dillingham,18.0,0.0,18.0,0.15,NaN,5000.0,3278.0,...,5.491,38.298,0.000,0.000,0.000,3.600,5.491,38.298,Facility aggregation,NaN
4,2090,AK,Fairbanks North Star,145.0,13.0,175.0,0.46,0.66,98971.0,72505.0,...,2.000,14.210,0.131,0.179,1.274,1.768,2.414,17.150,Facility aggregation,NaN


In [24]:
hosp_county.isna().sum()

fips_code                                        0
State                                            0
County Name                                      0
Staffed All Beds                                 0
Staffed ICU Beds                                 0
Licensed All Beds                                0
All Bed Occupancy Rate                          77
ICU Bed Occupancy Rate                        1127
Population                                       0
Population (20+)                                 0
Population (65+)                                 0
Staffed All Beds [Per 1000 People]              28
Staffed All Beds [Per 1000 Adults (20+)]        28
Staffed All Beds [Per 1000 Elderly (65+)]       28
Staffed ICU Beds [Per 1000 People]              28
Staffed ICU Beds [Per 1000 Adults (20+)]        28
Staffed ICU Beds [Per 1000 Elderly (65+)]       28
Licensed All Beds [Per 1000 People]             28
Licensed All Beds [Per 1000 Adults (20+)]       28
Licensed All Beds [Per 1000 Eld

In [25]:
df = df.merge(hosp_county, left_index=True, right_on='fips_code')

In [26]:
df.corr()['poverty_rate'].sort_values(ascending=True).head(20)

median_income                                                 -0.746680
income_per_capita                                             -0.711901
median_rent                                                   -0.425540
owner_occupied_housing_units_lower_value_quartile             -0.424752
owner_occupied_housing_units_median_value                     -0.403994
renter_occupied_housing_units_paying_cash_median_gross_rent   -0.391434
owner_occupied_housing_units_upper_value_quartile             -0.370109
median_age                                                    -0.233503
white_male_45_54                                              -0.176139
white_male_55_64                                              -0.169933
white_pop                                                     -0.166289
income_150000_199999                                          -0.153191
income_125000_149999                                          -0.146695
income_200000_or_more                                         -0

In [27]:
income_cols = [col for col in df.columns if 'income' in col]
df.drop(columns=income_cols, inplace=True)

In [28]:
df.corr()['poverty_rate'].sort_values(ascending=False).head(20)

poverty_rate                                                   1.000000
gini_index                                                     0.591470
death_rate                                                     0.277568
Staffed ICU Beds [Per 1000 Elderly (65+)]                      0.115113
Staffed ICU Beds [Per 1000 Adults (20+)]                       0.111187
Staffed All Beds [Per 1000 Elderly (65+)]                      0.109685
Staffed ICU Beds [Per 1000 People]                             0.107673
Licensed All Beds [Per 1000 Elderly (65+)]                     0.103327
amerindian_pop                                                 0.100106
mobile_homes                                                   0.074932
Licensed All Beds [Per 1000 Adults (20+)]                      0.065482
Licensed All Beds [Per 1000 People]                            0.057990
Staffed All Beds [Per 1000 Adults (20+)]                       0.057538
Staffed All Beds [Per 1000 People]                             0

In [29]:
cols_to_drop = ['median_rent',
                'owner_occupied_housing_units_lower_value_quartile',
                'owner_occupied_housing_units_median_value',
                'renter_occupied_housing_units_paying_cash_median_gross_rent',
                'owner_occupied_housing_units_upper_value_quartile',
                'pop_determined_poverty_status',
                'poverty', 'aggregate_travel_time_to_work']

df.drop(columns=cols_to_drop, inplace=True)

In [30]:
df.isna().sum().sort_values(ascending=False).head(30)

pop_divorced                                      2504
pop_5_years_over                                  2504
pop_widowed                                       2504
pop_separated                                     2504
pop_now_married                                   2504
speak_spanish_at_home_low_english                 2504
speak_spanish_at_home                             2504
speak_only_english_at_home                        2504
pop_never_married                                 2504
pop_15_and_over                                   2504
ICU Bed Source Last Updated                       2502
ICU Bed Occupancy Rate                            1115
All Bed Occupancy Rate                              73
commuters_16_over                                    1
commute_45_59_mins                                   1
commute_10_14_mins                                   1
commute_35_44_mins                                   1
commute_30_34_mins                                   1
walked_to_

In [31]:
for col in df.columns:
    if df[col].isna().sum() > 100:
        df.drop(columns=col, inplace=True)

# "Clean" DF here after the drop?

In [32]:
df.dropna(inplace=True)

In [33]:
x_cols = ['death_rate', 'Staffed ICU Beds [Per 1000 People]',]
X = df.select_dtypes(np.number).drop(columns='poverty_rate')
y = df['poverty_rate']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42)

ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [35]:
X.isna().sum()

confirmed_cases                               0
deaths                                        0
total_pop                                     0
households                                    0
male_pop                                      0
                                             ..
Staffed ICU Beds [Per 1000 Adults (20+)]      0
Staffed ICU Beds [Per 1000 Elderly (65+)]     0
Licensed All Beds [Per 1000 People]           0
Licensed All Beds [Per 1000 Adults (20+)]     0
Licensed All Beds [Per 1000 Elderly (65+)]    0
Length: 222, dtype: int64

In [36]:
df.shape

(2430, 231)

In [238]:
y.isna().sum()

0

### Linear Regression

In [239]:
lr = LinearRegression()
cv_score = cross_val_score(lr, X_train, y_train)
print(cv_score.mean(), cv_score.std())
cv_score

-0.022824012094294587 0.3635910147692979


array([ 0.32444544, -0.40867518,  0.18001054, -0.51536704,  0.30546618])

scaled  0.31177659, 0.34379026, 0.30268388, 0.30058899, 0.35387436]


In [260]:
random_test = ['a', 'b', 'a', 'b', 'a', 'b']

In [261]:
random_test.remove('a')

In [262]:
random_test

['b', 'a', 'b', 'a', 'b']

In [259]:
X_train.shape

(1822, 222)

In [258]:
lasso = Lasso(max_iter = 10000, alpha=.5, tol = 0.001)
cv_score = cross_val_score(lasso, X_train, y_train)
print(cv_score.mean(), cv_score.std())
cv_score

C:\Users\098uy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.5730186387033471, tolerance: 0.005718723584455529
  model = cd_fast.enet_coordinate_descent(
C:\Users\098uy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.6672518133650556, tolerance: 0.005655798372327986
  model = cd_fast.enet_coordinate_descent(
C:\Users\098uy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.535344823937416, tolerance: 0.0056407148995016336
  model = cd_fast.enet_coordinate_descent(
C:\Users\098uy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Conve

-0.5314395752298764 1.191494831141953


C:\Users\098uy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.4327212344856242, tolerance: 0.005785896696319561
  model = cd_fast.enet_coordinate_descent(


array([ 0.26083465, -0.28151675,  0.18154249,  0.06736336, -2.88542161])

### Support Vector Regressor

In [78]:
svr = SVR()

params = {
    'kernel': ['rbf', 'poly', 'sigmoid', 'linear'],
    'gamma': ['scale', 'auto'],
    'C': [.001, .01, .1, 1]
}

gs = GridSearchCV(svr, param_grid=params, verbose=3, n_jobs= -1)
gs.fit(X_train_sc, y_train)
print(gs.score(X_test_sc, y_test))
gs.best_params_

Fitting 5 folds for each of 32 candidates, totalling 160 fits
0.2696457990858857


{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}

### Random Forest Regressor

In [80]:
rfr = RandomForestRegressor()

params = {
    'n_estimators': [75, 100, 125],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 5],
}

gs = GridSearchCV(rfr, param_grid=params, verbose=10, n_jobs= -1)
gs.fit(X_train_sc, y_train)
print(gs.score(X_test_sc, y_test))
gs.best_params_

Fitting 5 folds for each of 54 candidates, totalling 270 fits
0.34513800558524654


{'max_depth': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 75}

### AdaBoost Regressor

In [81]:
ada = AdaBoostRegressor()

params = {
    'n_estimators': [40, 50, 70, 100],
    'learning_rate': [1, .1, .01, .001],
    'loss': ['linear', 'square', 'exponential']
}

gs = GridSearchCV(ada, param_grid=params, verbose=3, n_jobs= -1)
gs.fit(X_train_sc, y_train)
print(gs.score(X_test_sc, y_test))
gs.best_params_

Fitting 5 folds for each of 48 candidates, totalling 240 fits
0.3364917185371403


{'learning_rate': 0.001, 'loss': 'exponential', 'n_estimators': 100}

### Gradient Boosting Regressor

In [82]:
gbr = GradientBoostingRegressor()

params = {
    'loss': ['ls', 'lad', 'huber'],
    'learning_rate': [.1, .01, .001],
    'n_estimators': [90, 100, 120],
    'max_depth': [2, 3, 4]
}

gs = GridSearchCV(gbr, param_grid=params, verbose=3, n_jobs= -1)
gs.fit(X_train_sc, y_train)
print(gs.score(X_test_sc, y_test))
gs.best_params_

Fitting 5 folds for each of 81 candidates, totalling 405 fits
0.3376142279713219


{'learning_rate': 0.1, 'loss': 'lad', 'max_depth': 2, 'n_estimators': 90}